<sub>Developed by SeongKu Kang, August 2025 — Do not distribute</sub>

# 📘 Task 1: Product category Classification (BERT Fine-tuning version)

In this notebook, we use **BERT** to obtain contextual representations of product texts, taking the `[CLS]` token embedding as a compact summary of the input.  
A simple **linear layer classifier** is then trained on top of these embeddings to predict product categories.  

This approach leverages semantic knowledge from large-scale pretraining, allowing us to move beyond surface-level word overlap and capture deeper contextual meaning in product descriptions.

⚠️ **Note**   
This notebook may take a long time to run, since it repeatedly encodes texts with BERT.  
It is provided mainly for **reference**, and you are encouraged to review the workflow rather than execute every cell.

In [ ]:
import json
from tqdm import tqdm
from pathlib import Path
from utils import * 
import copy

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split, ConcatDataset

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
# Default paths
ROOT = Path("dataset") # Root dataset directory
CORPUS_PATH = ROOT / "corpus.jsonl" # Product corpus file (JSON Lines): Each line contains a product ID and its associated text description.

# Task 1: Product category classification
LABEL_MAP_PATH = ROOT / "category_classification" 
LABEL2ID_PATH = LABEL_MAP_PATH / "label2labelid.json" 
ID2LABEL_PATH = LABEL_MAP_PATH / "labelid2label.json" 
PID2LABEL_TRAIN_PATH = LABEL_MAP_PATH / "pid2labelids_train.json"
PID2LABEL_TEST_PATH = LABEL_MAP_PATH / "pid2labelids_test.json" 

# === Load limited label training set (subset of pid2labelids_train) ===
PID2LABEL_TRAIN_LIMITED_PATH = LABEL_MAP_PATH / "pid2labelids_train_limited.json" 

In [3]:
pid2text = load_corpus(CORPUS_PATH) # load corpus

label2id = load_json(LABEL2ID_PATH)
id2label = load_json(ID2LABEL_PATH)
pid2label_train = load_json(PID2LABEL_TRAIN_PATH)
pid2label_test = load_json(PID2LABEL_TEST_PATH)

pid2label_train_limited = load_json(PID2LABEL_TRAIN_LIMITED_PATH)

In [4]:
class ProductCategoryBertDataset(Dataset):
    """
    A PyTorch Dataset for product category classification using BERT embeddings.

    Args:
        pid2label (dict): Mapping from product ID to its label (category index).
        pid2text (dict): Mapping from product ID to its raw text (title + description).
        tokenizer (transformers.PreTrainedTokenizer): BERT tokenizer for encoding texts.
        max_length (int, optional): Maximum sequence length for tokenization. Default is 512.

    Attributes:
        pids (list): List of product IDs in the dataset.
        labels (list): List of label indices corresponding to each product.
        texts (list): List of raw product texts.
        tokenizer: Tokenizer used for encoding texts.
        max_length: Maximum sequence length for padding/truncation.

    Returns (in __getitem__):
        dict: A dictionary containing:
            - "input_ids" (torch.LongTensor): Tokenized IDs of the product text
            - "attention_mask" (torch.LongTensor): Attention mask for padding
            - "y" (torch.LongTensor): Label index
    """

    def __init__(self, pid2label, pid2text, tokenizer, max_length=512):
        self.pids = list(pid2label.keys())
        self.labels = [pid2label[pid] for pid in self.pids]
        self.texts = [pid2text[pid] for pid in self.pids]
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.pids)

    def __getitem__(self, idx):
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        encoded = self.tokenizer(
            self.texts[idx],
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            "input_ids": encoded["input_ids"].squeeze(0),
            "attention_mask": encoded["attention_mask"].squeeze(0),
            "y": label
        }

In [5]:
from sklearn.metrics import accuracy_score, f1_score

def evaluate(model, dataloader, device="cpu"):
    """
    Evaluate a BERT-based classification model on a given dataset.

    Args:
        model (torch.nn.Module): The classification model to evaluate.
        dataloader (DataLoader): DataLoader providing batches with 
            - "input_ids": tokenized product text IDs
            - "attention_mask": attention masks for padding
            - "y": ground-truth labels
        device (str, optional): Device to run evaluation. 
    """
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            y = batch["y"].to(device)

            logits = model(input_ids=input_ids, attention_mask=attention_mask)
            preds = torch.argmax(logits, dim=1)

            all_preds.extend(preds.cpu().tolist())
            all_labels.extend(y.cpu().tolist())

    acc = accuracy_score(all_labels, all_preds)
    f1_macro = f1_score(all_labels, all_preds, average="macro", zero_division=0)

    return {"accuracy": acc, "f1_macro": f1_macro}

In [6]:
import torch.nn as nn
from transformers import AutoModel

class BertClassifier(nn.Module):
    """
    A BERT-based classifier that uses the [CLS] token embedding for classification.

    Args:
        num_classes (int): Number of output classes.
        model_name (str, optional): Name of the pre-trained BERT model. Default is "bert-base-uncased".
        dropout (float, optional): Dropout rate before the classification layer. Default is 0.1.
        freeze_bert (bool, optional): If True, freeze BERT parameters and only train the classifier. Default is True.

    Attributes:
        bert (transformers.AutoModel): Pre-trained BERT encoder.
        classifier (nn.Sequential): A simple feed-forward classifier (Dropout + Linear).

    Forward Input:
        input_ids (torch.LongTensor): Tokenized input IDs of shape (batch_size, seq_len).
        attention_mask (torch.LongTensor): Attention masks of shape (batch_size, seq_len).

    Forward Output:
        torch.Tensor: Logits of shape (batch_size, num_classes).
    """

    def __init__(self, num_classes, model_name="bert-base-uncased", dropout=0.1, freeze_bert=True):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        hidden_size = self.bert.config.hidden_size
        self.classifier = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(hidden_size, num_classes)
        )

        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.last_hidden_state[:, 0]  # [CLS] token embedding
        logits = self.classifier(cls_output)
        return logits  # (batch_size, num_classes)

In [7]:
# === Prepare tokenizer and test dataset ===
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

test_dataset = ProductCategoryBertDataset(pid2label_test, pid2text, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=64)

# === Define model dimensions ===   
num_classes = len(label2id)

# === Manage product ID sets ===
all_pids = set(pid2text.keys())
test_pids = set(pid2label_test.keys())

In [8]:
results_dict = {'valid':{}, 'test': {}}

In [9]:
# Create training dataset
train_dataset = ProductCategoryBertDataset(pid2label_train, pid2text, tokenizer)

# Split into train/validation (80/20)
val_ratio = 0.2
val_size = int(len(train_dataset) * val_ratio)
train_size = len(train_dataset) - val_size
train_split, val_split = random_split(train_dataset, [train_size, val_size])

# DataLoaders
train_loader = DataLoader(train_split, batch_size=128, shuffle=True)  # shuffle for training
val_loader = DataLoader(val_split, batch_size=64)  # no shuffle for validation

In [10]:
# Identify unlabeled product IDs
used_pids = set(pid2label_train.keys())      # products with labels in training set
unlabeled_pids = all_pids - used_pids        # remaining products without labels
unlabeled_ratio = len(unlabeled_pids) / len(all_pids)

# Print statistics
print(f"[Unlabeled] {len(unlabeled_pids)} / {len(all_pids)} = {unlabeled_ratio:.2%}")

[Unlabeled] 26167 / 39452 = 66.33%


In [11]:
model = BertClassifier(num_classes).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4)

In [ ]:
best_val_acc = -1
best_model_state = None
patience = 5
patience_counter = 0

val_acc_list = []
test_acc_list = []

EPOCHS = 500

for epoch in range(1, EPOCHS + 1):
    model.train()
    total_loss = 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch}"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        logits = model(input_ids=input_ids, attention_mask=attention_mask)

        y = batch["y"].to(device)
        loss = F.cross_entropy(logits, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"[Epoch {epoch}] Train Loss: {avg_loss:.4f}")

    # === Validation ===
    val_result = evaluate(model, val_loader, device=device)
    val_acc = val_result["accuracy"]
    val_acc_list.append(val_acc)

    is_improved = val_acc > best_val_acc
    print_eval_result(val_result, stage="val", is_improved=is_improved)

    # === Test ===
    test_result = evaluate(model, test_loader, device=device)
    test_acc = test_result["accuracy"]
    test_acc_list.append(test_acc)
    print_eval_result(test_result, stage="test")

    # === Update best model ===
    if is_improved:
        best_val_acc = val_acc
        best_model_state = copy.deepcopy(model.state_dict())
        patience_counter = 0
    else:
        patience_counter += 1

    # === Early stopping ===
    if patience_counter >= patience:
        print(f"[Early Stopping] No improvement for {patience} consecutive epochs.")
        break

Epoch 1: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 84/84 [01:32<00:00,  1.10s/it]


[Epoch 1] Train Loss: 6.6472
[VAL ] Acc: 0.0433 | F1-macro: 0.0014 *
[TEST] Acc: 0.0440 | F1-macro: 0.0012


Epoch 2: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 84/84 [01:26<00:00,  1.04s/it]


[Epoch 2] Train Loss: 6.2842
[VAL ] Acc: 0.0531 | F1-macro: 0.0028 *
[TEST] Acc: 0.0533 | F1-macro: 0.0022


Epoch 3: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 84/84 [01:27<00:00,  1.04s/it]


...

Epoch 74: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 84/84 [01:27<00:00,  1.04s/it]


[Epoch 74] Train Loss: 1.3808
[VAL ] Acc: 0.5269 | F1-macro: 0.4147 *
[TEST] Acc: 0.5429 | F1-macro: 0.4640


Epoch 75: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 84/84 [01:27<00:00,  1.04s/it]


[Epoch 75] Train Loss: 1.3498
[VAL ] Acc: 0.5265 | F1-macro: 0.4154
[TEST] Acc: 0.5440 | F1-macro: 0.4651


Epoch 76:  43%|█████████████████████████████████████████████████████████████████████████▎                                                                                                 | 36/84 [00:37<00:50,  1.05s/it]

In [ ]:
model.load_state_dict(best_model_state)
final_test_result = evaluate(model, test_loader, device=device)
print_eval_result(final_test_result, stage="final_test")

results_dict['valid']['bert_train'] = val_acc_list[:]
results_dict['test']['bert_train'] = test_acc_list[:]

In [ ]:
plot_results(results_dict, split="valid")
plot_results(results_dict, split="test")